In [0]:
SPARK - Bronze - The code you have with spark.write.format("delta").mode("append").saveAsTable(...) is just a Spark job, not a DLT pipeline.

from pyspark.sql.functions import current_timestamp
import requests

CATALOG = "bronze"  

# Fully qualified table names
API_TBL = f"{CATALOG}.bronze_api_data"
ADLS_TBL = f"{CATALOG}.bronze_adls_data"
VOLUME_TBL = f"{CATALOG}.bronze_volumes_data"


# API Source → Bronze

url = "https://api.example.com/data"
response = requests.get(url)
data = response.json()
df_api = spark.createDataFrame(data).withColumn("ingestion_date", current_timestamp())
df_api.write.format("delta").mode("append").saveAsTable(API_TBL)


# ADLS Source → Bronze

df_adls = (
    spark.read.format("parquet")
    .load("abfss://container@storageaccount.dfs.core.windows.net/folder/")
    .withColumn("ingestion_date", current_timestamp())
)
df_adls.write.format("delta").mode("append").saveAsTable(ADLS_TBL)


# Volumes / Local File → Bronze

df_volume = (
    spark.read.format("csv")
    .option("header", "true")
    .load("/mnt/volumes/data.csv")
    .withColumn("ingestion_date", current_timestamp())
)
df_volume.write.format("delta").mode("append").saveAsTable(VOLUME_TBL)
